In [ ]:
JIRA_DOMAIN = "gremlininc.atlassian.net"

In [13]:
import netrc

auth_file = netrc.netrc()
authTokens = auth_file.authenticators(JIRA_DOMAIN)

user_email = authTokens[0]
api_token = authTokens[2]

In [15]:
from jira import JIRA

jira_url = f"https://{JIRA_DOMAIN}/"

jira = JIRA(jira_url, basic_auth=(user_email, api_token))

# projects = jira.projects()  # Test Authentication